$\newcommand{\vect}[1]{\vec{#1}}
\newcommand{\uvect}[1]{\hat{#1}}
\newcommand{\abs}[1]{\lvert#1\rvert}
\newcommand{\norm}[1]{\lVert#1\rVert}
\newcommand{\I}{\mathrm{i}}
\newcommand{\ket}[1]{\left|#1\right\rangle}
\newcommand{\bra}[1]{\left\langle#1\right|}
\newcommand{\braket}[1]{\langle#1\rangle}
\newcommand{\op}[1]{\mathbf{#1}}
\newcommand{\mat}[1]{\mathbf{#1}}
\newcommand{\d}{\mathrm{d}}
\newcommand{\pdiff}[3][]{\frac{\partial^{#1} #2}{\partial {#3}^{#1}}}
\newcommand{\diff}[3][]{\frac{\d^{#1} #2}{\d {#3}^{#1}}}
\newcommand{\ddiff}[3][]{\frac{\delta^{#1} #2}{\delta {#3}^{#1}}}
\DeclareMathOperator{\erf}{erf}
\DeclareMathOperator{\order}{O}
\DeclareMathOperator{\diag}{diag}$

#Random Non-linear Schrödinger Equation

Here is a small test-problem evolving a Schrödinger-like equation of the form

$$
  \I\ket{\dot{\psi}} = \mat{H}\ket{\psi}
$$

where the Hamiltonian depends non-linearly on $\ket{\psi}$.  We apply to this equation an external potential.

In [ ]:
from __future__ import division
import sys;sys.path.append('../../')

import numpy as np

from pytimeode import interfaces
from mmfutils import interface

class Problem(object):
    def __init__(self, N=5, seed=1, g=1.0):
        np.random.seed(seed)
        H = np.random.rand(N, N) + np.random.rand(N, N)*1j - 0.5 - 0.5j
        H = H + H.conj().T
        
        V = np.random.rand(N, N) + np.random.rand(N, N)*1j - 0.5 - 0.5j
        V = V + V.conj().T
        
        Es, Psi = np.linalg.eigh(H)
        H -= Es[0]*np.eye(N)
        Es -= Es[0]
        psi0 = Psi[:, 0]
        H -= g*np.diag(abs(psi0)**2)
        
        self.H = H
        self.V = V
        self.g = g
        self.N = N

    def Vt(self, t):
        return 0.0*self.V
    
    def get_Hy(self, y, t):
        V = self.Vt(t) + self.g*abs(y)**2
        return (self.H + V).dot(y)

    
class State(interfaces.ArrayStateMixin):
    interface.implements(interfaces.IStateForABMEvolvers)
    
    def __init__(self, p):
        self.t = 0.0
        self.p = p
        self.data = np.empty(p.N, dtype=complex)
        
    def compute_dy(self, t, dy=None, potentials=None):
        if dy is None:
            dy = self.copy()
        dy.data[...] = self.p.get_Hy(y=self.data, t=t)
        dy *= -1j
        return dy
        
interface.verifyClass(interfaces.IStateForABMEvolvers, State)

In [ ]:
p = Problem()
y = State(p)
y.data[...] = 1.0
from pytimeode import evolvers
with y.lock:
    evolver = evolvers.EvolverABM(y=y, dt=0.01)
    evolver.evolve(200)
with y.lock:
    evolver = evolvers.EvolverABM(y=y, dt=0.01)
    evolver.evolve(2)
evolver.y

#Gross-Pitaevskii Equation (GPE) 

##1D 

In [ ]:
from __future__ import division
import sys;sys.path.append('../../')
import numpy as np
from pytimeode import interfaces
from mmfutils import interface

Here we briefly demonstrate using the framework to solve the GPE:

$$
  \I\hbar\pdiff{}{t}\psi(x,t) 
  = \left(\frac{-\hbar^2 \nabla^2}{2m} + g\abs{\psi(x,t)}^2 + V(x,t)\right)\psi(x,t).
$$

The state here is simply a complex array representing the wavefunction $\psi(x,t)$ so we can use the ``ArrayStateMixin``.  We represent the problem on an $N$-point lattice in a periodic box of length $L$ so we can use fourier methods.

As a test problem, we will look at a domain wall oscillating in a harmonic trap

$$
  V(x, t) = \frac{m\omega^2}{2}x^2 - \mu.
$$

The length-scales for this problem are the trap-length $a = \sqrt{\hbar/m\omega}$ and the inter-particle separation $1/\abs{\psi}^2$ and the interaction length-scale $l_g = \hbar^2/mg$.  If we assume that the Thomas-Fermi approximation is valid in the center of the trap, then the central density $n_0 = \mu/g$ is set by the chemical potential.  We choose units so that $\hbar = m = \omega = 1$, set the TF radius $R_{TF} = 8a$ so that $\mu = m\omega^2 R_{TF}^2/2 = 32 m\omega^2 a^2$.  Finally, we choose $g=\hbar^2/ma$ so that $l_g = a$.

In [ ]:
fft = np.fft.fft
ifft = np.fft.ifft

class State(interfaces.ArrayStateMixin):
    interface.implements(interfaces.IStateForABMEvolvers)
    
    def __init__(self, N=256, L=20.0, mu=32.0, hbar=1.0, m=1.0, g=1.0, omega=1.0,
                 cooling_phase=1.0,  # Make this 1j for imaginary time evolution
                 data=None,
            ):
        self.t = 0.0
        self.N = N
        self.L = L
        self.hbar = hbar
        self.m = m
        self.g = g
        self.mu = mu
        self.omega = omega
        self.cooling_phase = cooling_phase
        
        self.data = np.empty(self.N, dtype=complex)
        if data is not None:
            self.data[...] = data
        self.init()
        
        if data is None:
            # Use TF profile as initial state
            self.data[...] = np.sqrt(self.n_TF())
        
    def init(self):
        self.dx = self.L / self.N
        
        # Abscissa centered about zero
        self.x = np.arange(self.N)*self.dx - self.L/2.0
        
        # Momenta
        self.k = 2*np.pi * np.fft.fftfreq(self.N, d=self.dx)
        self.k2 = self.k**2
        
    def braket(self, a, b):
        return a.conj().dot(b)*self.dx
    
    def n_TF(self):
        """Return the TF density."""
        V = self.Vt(t=0)
        n_TF = np.where(self.mu > V, 
                        (self.mu - V)/self.g,
                        0.0)
        return n_TF

    def laplacian(self, psi):
        """Return the laplacian of `f(x)`."""
        return - ifft(self.k2 * fft(psi))
    
    def Vt(self, t):
        """Overload this to define a time-dependent external potential."""
        return self.m/2.0 * (self.omega*self.x)**2
    
    def get_V(self, psi, t=0.0):
        """Return the effective potential."""
        return self.g * abs(psi)**2 - self.mu + self.Vt(t=t)
        
    def apply_H(self, psi, t=0.0):
        hbar2m = self.hbar**2/2.0/self.m
        V = self.get_V(psi=psi, t=t)
        return -hbar2m * self.laplacian(psi) + V * psi
        
    def compute_dy(self, t, dy=None, potentials=None, subtract_mu=True):
        if dy is None:
            dy = self.copy()
        y = self.data.copy()
        Hy = self.apply_H(psi=self.data, t=t)

        if subtract_mu:
            # Subtract the chemical potential - ensures that evolution
            # is unitary even with complex time.
            mu = self.braket(y, Hy)/self.braket(y,y)
            Hy -= mu * y
            dy.data[...] = Hy
            
        dy *= (-1j/self.cooling_phase)
        return dy
    
    ##################################################################
    # Helpers
    # These are not needed for evolution, but are useful for analysis.
    def get_energy(self, t=0.0):
        """Return the energy of the state."""
        hbar2m = self.hbar**2/2.0/self.m
        psi = self.data
        n = psi.conj() * psi
        V = self.get_V(psi, t=t)
        e = -hbar2m * psi.conj()*self.laplacian(psi) + V*n + self.g * n**2 / 2
        E = e.sum() * self.dx
        assert np.allclose(E.imag, 0)
        return E.real

    def get_N(self):
        """Return the particle number of the state"""
        psi = self.data        
        return (abs(psi)**2).sum() * self.dx
    
    def normalize(self, N):
        self.data *= np.sqrt(N/self.get_N())
        
    def plot(self, t=0.0):
        from matplotlib import pyplot as plt
        psi = self.data
        n = abs(psi)**2
        phase = np.angle(psi*np.exp(1j*np.pi/2))
        plt.plot(y0.x, n)
        plt.xlabel('x')
        plt.ylabel('n')
        plt.twinx()
        plt.plot(y0.x, phase)
        plt.ylabel('phase')
        E = self.get_energy(t=t)
        N = self.get_N()
        plt.title("E={}, N={}".format(E, N))

interface.verifyClass(interfaces.IStateForABMEvolvers, State)

In [ ]:
import scipy.optimize
sp = scipy

def solve(self, t=0, solver=sp.optimize.newton_krylov, 
          verbose=False, callback=None):
    """Find the nearest stationary state using a non-linear solver."""
    psi0 = self.data.copy()
    
    def pack(psi):
        """Return 1D real vector x from psi."""
        return psi.view(dtype=float)

    def unpack(x):
        """Return complex state psi from x."""
        return x.view(dtype=complex)

    N = self.get_N()
    
    x0 = pack(psi=psi0)
    def f(x):
        self.data[...] = unpack(x=x)
        self.normalize(N=N)
        assert np.allclose(N, self.get_N())
        if callback is not None:
            callback(self)
            
        if verbose:
            print("E={}".format(self.get_energy()))            
        dy = self.compute_dy(t=t, subtract_mu=True)
        return pack(psi=dy.data)
    
    x = solver(f, x0)
    self.data[...] = unpack(x=x)

State.solve = solve

Here we start from the Thomas Fermi approximate solution:

In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt
y0 = State()
y0.data[...] = np.sqrt(y0.n_TF())
y0.plot()

Now we find the ground state and a state with a domain wall using the scipy non-linear solvers.  These are not very efficient and could definitely make use of the gradient information better, but they work to high precision.

In [ ]:
from IPython.display import display, clear_output
fig = plt.gcf()

def callback(y, n=[0]):
    if n[0] % 1000 == 0:
        plt.clf()
        y.plot()
        clear_output(wait=True)
        display(fig)
    n[0] += 1

callback = None  # Just use it for debugging

y1 = y0.copy()
%time y1.solve(callback=callback)
y1.plot()
display(plt.gcf())

dw0 = y0.copy()
dw0.data[...] = np.sqrt(dw0.n_TF())*np.sign(dw0.x)
dw0.normalize(N=y0.get_N())  # Normalize to same particle number.
dw1 = dw0.copy()
%time dw1.solve(callback=callback)
plt.close('all')
dw1.plot()
display(plt.gcf())

Now we cool with imaginary time evolution to the ground state:

In [ ]:
from IPython.display import display, clear_output
from pytimeode import evolvers
fig = plt.gcf()
y0 = State()
y0.data[...] = np.sqrt(y0.n_TF())

y0.cooling_phase = 1j
with y0.lock:
    evolver = evolvers.EvolverABM(y=y0, dt=0.0005)
    for n in xrange(12):
        evolver.evolve(1000)
        plt.clf()
        y = evolver.get_y()
        y.plot(t=evolver.t)
        clear_output(wait=True)
        display(fig)
    plt.close('all')

y0 = evolver.get_y()

Here we imprint a phase flip and cool to the lowest energy state which is a dark soliton at rest in the core of the trap.  Numerical noise allows the evolution the break the symmetry and the soliton - a saddle point - is bypassed, ending up again in the ground state.

In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt
y0 = State()
y0.data[...] = np.sqrt(y0.n_TF())*np.sign(y0.x)

fig = plt.gcf()
y0.cooling_phase = 1j
N = y0.get_N()
with y0.lock:
    evolver = evolvers.EvolverABM(y=y0, dt=0.0001)
    for n in xrange(20):
        evolver.evolve(2000)
        plt.clf()
        y = evolver.get_y()
        
        # Constrain solution to be real
        #y.data[...] = y.data.real
        #y.data *= np.sqrt(N/y.get_N())
        y.plot(t=evolver.t)
        clear_output(wait=True)
        display(fig)
    plt.close('all')

# Dimer Method

In [ ]:
def dot(self, a):
    """Return the inner product of two states."""
    return np.dot(self.data.conj().ravel(), a.data.ravel())

def norm(self):
    """Return the norm of the state."""
    return np.sqrt(self.dot(self))

def get_force(self, t=0.0):
    """Return the force on the state at the current position."""
    # Assume for now the state is real:
    assert np.allclose(self.data.imag, 0)
    F = -1j*self.compute_dy(t=t, subtract_mu=True)  # -Hy
    assert np.allclose(F.data.imag, 0)
    return F

def proj(self, x):
    """Project x along the current directions"""
    return self * self.dot(x)/self.norm()

def proj_perp(self):
    """Project x perpendicular to the current directions"""
    return self - self.proj(x)

State.dot = dot
State.norm = norm
State.get_force = get_force
State.proj = proj
State.proj_perp = proj_perp

class DimerMode(object):
    zero_torque = 0
    find_saddle = 1
    follow_saddle = 2

class Dimer(interfaces.MultiStateMixin):
    def __init__(self, y1, y2, mode):
        self.data = dict(
            R=(y1 + y2)/2, 
            r=(y1 - y2)/2)
        self.mode = mode
    
    @property
    def R(self):
        return self.data['R']

    @R.setter
    def R(self, value):
        self.data['R'] = value

    @property
    def r(self):
        return self.data['r']

    @r.setter
    def r(self, value):
        self.data['r'] = value

    @property
    def y1(self):
        return self.R + self.r

    @property
    def y2(self):
        return self.R - self.r
        
    def set_length(self, d):
        self.r *= d/norm(self.r)
    
    def get_torque(self):
        F1 = self.y1.get_force()
        F2 = self.y2.get_force()
        self.r.proj_perp(F1 - F2)

    def compute_dy(self, t, dy=None, potentials=None, subtract_mu=True):
        if dy is None:
            dy = self.copy()
        y = self.data.copy()
        Hy = self.apply_H(psi=self.data, t=t)

        if subtract_mu:
            # Subtract the chemical potential - ensures that evolution
            # is unitary even with complex time.
            mu = self.braket(y, Hy)/self.braket(y,y)
            Hy -= mu * y
            dy.data[...] = Hy
            
        dy *= (-1j/self.cooling_phase)
        return dy

        
    
y0 = State()
y1 = State() * 1.1
d = Dimer(y0, y1, mode=DimerMode.zero_torque)
d.set_length(0.1)
norm(d.r)